In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 20, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BD6aHbGGUl0yu57oWJf1BugRdr8Lw', 'finish_reason': 'stop', 'logprobs': None}, id='run-e56e526c-1323-403b-ac45-eab42e6ea6d2-0', usage_metadata={'input_tokens': 20, 'output_tokens': 4, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [4]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [5]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [6]:
response = model.invoke(prompt)
print(response.content)

Ciao!


In [7]:
for token in model.stream(messages):
    print(token.content, end="|")

|C|iao|!||

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatOllama(
    model="deepseek-r1:8b",
    temperature=0,
    # other params...
)

messages = [
    SystemMessage("You are a helpful assistant that translates English to French. Translate the user sentence."),
    HumanMessage("I love programming."),
]

# ai_message = llm.invoke(messages)



In [3]:
for token in llm.stream(messages):
    print(token.content, end="")

<think>
Okay, so I need to figure out how to translate "I love programming." into French. Let me start by breaking down each part of the sentence.

First, "I love" is straightforward. In English, it's "I love," which in French would be "J'adore." That makes sense because "j'adore" is a common way to express deep affection or enjoyment for something.

Next, the word "programming." In French, programming is called "le développement de logiciels" or more commonly just "le programmation." So, I can use either of those. Since it's about programming itself rather than software development specifically, maybe "le programmation" is better because it directly translates to "programming."

Putting it all together, the sentence would be "J'adore le programmation." That sounds natural and conveys the same sentiment as the original English statement.

I should also consider if there's a more colloquial way to say it. Sometimes people might use "Je aime" instead of "J'adore," but "j'adore" is strong

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


class LangChainLLM:
    def __init__(self, params: dict = None):
        if params is None:
            params = {}
        """Initialize the LLM with specified model and parameters."""
        
        # Check if model is specified in params, if not set default to llama3.1
        if "model" not in params:
            params["model"] = "llama3.1"
            print(f"Model not specified, defaulting to {params['model']}")

        # Define parameters in a dictionary for easier modification
        self.llm_params = params
        self.llm = ChatOllama(**self.llm_params)

        # Initialize memory to store conversation history
        self.memory = []
        self.system_message = None

    def invoke(self, messages):
        """Send a single request to the LLM and get response."""
        try:
            # Store messages in memory if they're not already there
            self._update_memory(messages)

            response = self.llm.invoke(messages)

            # Add the response to memory
            self._add_to_memory(response)

            return response
        except Exception as e:
            print(f"Error during invocation: {e}")
            return None

    def stream(self, messages):
        """Stream responses from the LLM."""
        try:
            # Store messages in memory before streaming
            self._update_memory(messages)

            # We'll collect the full response to add to memory after streaming
            return self.llm.stream(messages)
        except Exception as e:
            print(f"Error during streaming: {e}")
            return None

    def stream2(self, messages):
        output = ""
        for token in llm.stream(messages):
            output += str(token.content)
            yield (token)
        message = AIMessage(output)
        self._update_memory([message])

    def batch(self, message_list):
        """Process multiple message sets in batch."""
        try:
            # Note: Memory handling for batch operations is more complex
            # and might need custom implementation based on use case
            return self.llm.batch(message_list)
        except Exception as e:
            print(f"Error during batch processing: {e}")
            return None

    def generate_prompt(self, system_message, user_message, use_memory=True):
        """Helper to create properly formatted message list."""
        # Store the system message for future use
        if self.system_message is None:
            self.system_message = system_message

        if use_memory and self.memory:
            # Create a message list with system message and all conversation history
            messages = [SystemMessage(system_message)]
            messages.extend(self.memory)
            messages.append(HumanMessage(user_message))
            return messages
        else:
            # Return just the new messages without history
            return [SystemMessage(system_message), HumanMessage(user_message)]

    def _update_memory(self, messages):
        """Update memory with new messages if they're not already stored."""
        # Extract and store system message if present
        if messages and isinstance(messages[0], SystemMessage):
            self.system_message = messages[0].content

        # Add human and AI messages to memory, skipping system message
        for msg in messages:
            if not isinstance(msg, SystemMessage) and msg not in self.memory:
                self.memory.append(msg)

    def _add_to_memory(self, message):
        """Add a response message to memory."""
        if message and message not in self.memory:
            self.memory.append(message)

    def clear_memory(self):
        """Clear the conversation history."""
        self.memory = []

    def get_memory(self):
        """Return the current conversation history."""
        return self.memory


# Example usage:
llm = LangChainLLM(params={"model": "llama3.1"})
messages = llm.generate_prompt(
    "You are a helpful assistant.",
    "My name is Stavros and I love programming.",
)
for token in llm.stream2(messages):
    print(token.content, end="")

Nice to meet you, Stavros! It's great to hear that you're passionate about programming. What kind of programming languages or areas of development are you interested in? Web development, mobile app development, artificial intelligence, machine learning, or something else?

In [14]:
llm.get_memory()

[HumanMessage(content='My name is Stavros and I love programming.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Nice to meet you, Stavros! It's great that you're passionate about programming. What kind of programming languages or areas are you most interested in? Web development, mobile app creation, artificial intelligence, or something else?", additional_kwargs={}, response_metadata={})]

In [4]:
llm.invoke(llm.get_memory() + [HumanMessage('What is the capital of France? Also, what is my name?')])
llm.get_memory()


[HumanMessage(content='My name is Stavros and I love programming.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Nice to meet you, Stavros! It's great to hear that you're passionate about programming. What kind of programming languages or areas of development are you interested in? Web development, mobile app development, artificial intelligence, machine learning, or something else?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the capital of France? Also, what is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Two quick questions!\n\nThe capital of France is Paris.\n\nAnd... your name is Stavros!', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-03-20T09:35:25.695079Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2846446900, 'load_duration': 30571300, 'prompt_eval_count': 97, 'prompt_eval_duration': 859307600, 'eval_count': 21, 'eval_duration': 1955544900, 'mess

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm.llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='<think>\nOkay, so I need to translate the sentence "I love programming." into German. Let me think about how to approach this.\n\nFirst, I know that "I love" in English is usually translated as "Ich liebe" in German. So that part seems straightforward.\n\nNext, "programming" is the subject here. In German, the word for programming is "Programmieren." But wait, sometimes people might refer to it as just "Programmen," but I think "Programmieren" is more accurate when talking about the activity of programming rather than the noun form.\n\nPutting it together, "Ich liebe Programmieren." That makes sense. Let me double-check if there\'s a better way to phrase it. Maybe using "to love" instead of "I love"? But in this case, since it\'s about the person\'s feelings, "Ich liebe" is appropriate.\n\nAlso, I should consider the context. If someone says they love programming, it\'s about their passion for writing code or developing software. So "Programmieren" fits well here bec

In [13]:
from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama


@tool
def get_user_name(user_id: int, addresses: List[str])-> str:
    """Return user name using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return "Stavros Pitoglou"


llm = ChatOllama(
    model="llama3.1",
    temperature=0,
).bind_tools([get_user_name])

result = llm.invoke(
    "Could you bring me the name of the user with id 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX. The name should be the output of the get_user_name tool."
)
result.tool_calls

[{'name': 'get_user_name',
  'args': {'addresses': ['123 Fake St, Boston, MA',
    '234 Pretend Boulevard, Houston, TX'],
   'user_id': 123},
  'id': '35cb4bad-d829-48b2-9099-fde8eba3c192',
  'type': 'tool_call'}]

In [14]:
result.content

''

In [15]:
result

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-03-18T07:11:07.4654946Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4177865400, 'load_duration': 27764300, 'prompt_eval_count': 247, 'prompt_eval_duration': 604000000, 'eval_count': 40, 'eval_duration': 3545000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-187bc9f0-5bd7-43b5-a389-ba57b5699b29-0', tool_calls=[{'name': 'get_user_name', 'args': {'addresses': ['123 Fake St, Boston, MA', '234 Pretend Boulevard, Houston, TX'], 'user_id': 123}, 'id': '35cb4bad-d829-48b2-9099-fde8eba3c192', 'type': 'tool_call'}], usage_metadata={'input_tokens': 247, 'output_tokens': 40, 'total_tokens': 287})

In [12]:
get_user_name.args_schema.model_json_schema()

{'description': 'Return user name using historical addresses.\n\nArgs:\n    user_id (int): the user ID.\n    addresses (List[str]): Previous addresses as a list of strings.',
 'properties': {'user_id': {'title': 'User Id', 'type': 'integer'},
  'addresses': {'items': {'type': 'string'},
   'title': 'Addresses',
   'type': 'array'}},
 'required': ['user_id', 'addresses'],
 'title': 'get_user_name',
 'type': 'object'}

In [20]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

res = search.invoke("Obama's first name?")

In [25]:
from pprint import pprint
type(res)
pprint(str(res))


('The White House, official residence of the president of the United States, '
 'in July 2008. The president of the United States is the head of state and '
 'head of government of the United States, [1] indirectly elected to a '
 'four-year term via the Electoral College. [2] Under the U.S. Constitution, '
 'the officeholder leads the executive branch of the federal government and is '
 'the commander-in-chief of ... As the head of the government of the United '
 'States, the president is arguably the most powerful government official in '
 'the world. The president is elected to a four-year term via an electoral '
 'college system. Since the Twenty-second Amendment was adopted in 1951, the '
 'American presidency has been limited to a maximum of two terms.. Click on a '
 "president below to learn more about each presidency ... The Irish Sun, It's "
 "a fake Barack Obama's brother posts forged document he claims is "
 "ex-president's 'Kenyan birth certificate,' March 11, 2017 Salon, O

'T'

In [18]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

search.invoke("Obama")

"snippet: The former First Lady denied any rift with Barack Obama and praised school counselors in her first public video since speculation about their relationship. Learn about their love story, family and legacy in this article., title: Michelle Obama makes emotional announcement amid speculation over ..., link: https://www.hellomagazine.com/celebrities/812295/michele-obama-makes-emotional-announcement-amid-divorce-speculation/, snippet: Former Presidents Clinton, Obama, Biden and George W. Bush have barely uttered a word about President Trump's actions a month into his second term, to the dismay of Democratic critics who say…, title: Obama, Bush, Clinton, Biden staying silent on Trump dismays some Democrats, link: https://thehill.com/homenews/administration/5153858-former-presidents-trump-actions/, snippet: A viral claim suggests that Trump's allies proposed a constitutional amendment that would allow presidents to have a third term, including Obama. But the amendment only applies t

In [19]:
search = DuckDuckGoSearchResults(output_format="list")

search.invoke("Obama")

[{'snippet': 'Learn about the life and achievements of Barack Obama, the 44th president of the United States and the first African American to hold that office. Find out his biography, awards, books, family, and more.',
  'title': 'Barack Obama Facts | Britannica',
  'link': 'https://www.britannica.com/facts/Barack-Obama'},
 {'snippet': 'Obama, in his remarks, insisted that he is "convinced that if we want democracy as we understand it to survive," people must work for a renewed dedication to pluralist principles.',
  'title': "Obama, in 1st remarks since election, says 'a line has been ... - Fox News",
  'link': 'https://www.foxnews.com/politics/obama-first-remarks-since-election-says-a-line-has-been-crossed-one-side-makes-certain-moves'},
 {'snippet': 'Learn about the life and career of Barack Obama, the 44th president of the United States, who rose from humble beginnings to become a best-selling author, a U.S. senator, and a Nobel Peace Prize laureate. Explore his achievements, chal

In [3]:

from langchain_ollama import ChatOllama
from cssallmlib.implementations.llm_translation import TranslationChain

# Initialize the model and translation chain
llm = ChatOllama(model="llama3.1", temperature=0)
translator = TranslationChain(llm)

# Single translation
result = translator.translate("I love programming.", "German")
print(f"Translation: {result}")

# Batch translation
texts = ["Hello world!", "How are you?", "Good morning!"]
results = translator.batch_translate(texts, "Spanish")
print("\nBatch translations:")
for original, translated in zip(texts, results):
    print(f"{original} -> {translated}")

2025-03-20 14:32:03.431 | INFO     | cssallmlib.implementations.llm_translation:__init__:37 - Initialized TranslationChain with source language: English
2025-03-20 14:32:03.432 | INFO     | cssallmlib.implementations.llm_translation:translate:53 - Starting translation from English to German
2025-03-20 14:32:03.433 | DEBUG    | cssallmlib.implementations.llm_translation:translate:54 - Input text: I love programming.
2025-03-20 14:32:13.282 | INFO     | cssallmlib.implementations.llm_translation:translate:58 - Successfully translated text to German
2025-03-20 14:32:13.283 | DEBUG    | cssallmlib.implementations.llm_translation:translate:59 - Translated text: The translation of "I love programming" into German is:

"Ich liebe Programmieren."

Here's a breakd...
2025-03-20 14:32:13.284 | INFO     | cssallmlib.implementations.llm_translation:batch_translate:76 - Starting batch translation of 3 texts from English to Spanish


Translation: The translation of "I love programming" into German is:

"Ich liebe Programmieren."

Here's a breakdown of the translation:

* "Ich liebe" means "I love"
* "Programmieren" means "programming"

Note that in German, the verb "lieben" (to love) typically takes the dative case when referring to an activity or profession, which is why it's written as "Programmieren" instead of just "Programmierung".


2025-03-20 14:32:36.792 | INFO     | cssallmlib.implementations.llm_translation:batch_translate:83 - Successfully completed batch translation of 3 texts to Spanish



Batch translations:
Hello world! -> The translation of "Hello world!" into Spanish is:

"Hola mundo!"

(Note: This is a common phrase used in programming, especially when introducing a new language or environment. It's not typically used as a greeting in everyday conversation.)
How are you? -> The translation of "How are you?" into Spanish is:

**¿Cómo estás?**

(Note: This is a formal way to ask how someone is doing. If you want to use an informal tone, the translation would be **¿Cómo estás?** becomes **¿Qué onda?** or simply **¿Cómo estás?** in some Latin American countries.)
Good morning! -> The translation of "Good morning!" into Spanish is:

**Buenos días!**

(Note: In some Latin American countries, it's more common to say "Buenos días" until around 11 am or noon, after which you would switch to "Buenas tardes" for the afternoon and "Buenas noches" for the evening. However, in Spain, it's generally acceptable to use "Buenos días" throughout the morning.)
